"""
File: workout_planner_agent

Description: 

This file implements a LangGraph-based agent that utilizes custom workout planner tools to generate personalized fitness plans. Key components include:

1. Environment Setup:
   - Configures LangChain tracing and project settings
   - Manages API keys for OpenAI and LangChain

2. Tool Integration:
   - Incorporates DuckDuckGoSearchRun, WorkoutPlannerTool, and LlmWorkoutPlannerTool
   - Binds tools to a ChatOpenAI model (GPT-4)

3. Agent State and Workflow Definition:
   - Defines AgentState using TypedDict for managing conversation history
   - Creates a StateGraph workflow with agent and action nodes
   - Implements logic for conditional edges to determine workflow progression

4. Execution Functions:
   - run_workflow: Asynchronous function to stream workflow updates
   - execute_workflow: Synchronous wrapper for running the workflow
   - pretty_print_result: Formats and displays the final workout plan

5. LangGraph Integration:
   - Utilizes LangGraph components like StateGraph and ToolNode
   - Compiles the workflow into an executable application

This script orchestrates the entire process of generating a workout plan, from user input to final output, leveraging AI-powered tools and a sophisticated graph-based workflow. It demonstrates the integration of custom tools with language models in a flexible, extensible architecture suitable for complex, multi-step AI tasks.

- Author: Juan Olano
- Date created: 8/30/2024
- Last modified: 8/30/2024

This file is part of the Workout Planner Agent project.

License: MIT License

Copyright (c) {current_year} {your_name}

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

In [11]:
import os
from typing import TypedDict, Annotated
from uuid import uuid4

from langchain_openai import ChatOpenAI
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages

import asyncio
import nest_asyncio
nest_asyncio.apply()

# Import our custom workout planner tool
from workout_planner_tool import WorkoutPlannerTool
from workout_planner_llm_based_tool import LlmWorkoutPlannerTool

In [12]:
# Setup environment
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Workout Planner - {uuid4().hex[0:8]}"


import os
import openai

with open('../../../apikeys/api_openai_aimakerspace.key', 'r') as file:
    api_key = file.read().strip()

openai.api_key = api_key
os.environ["OPENAI_API_KEY"] = openai.api_key

import os

with open('../../../apikeys/api_langchain.key', 'r') as file:
    api_langchain_key = file.read().strip()
    
os.environ["LANGCHAIN_API_KEY"] = api_langchain_key

In [13]:
# Define tools
tool_belt = [
    DuckDuckGoSearchRun(),
    WorkoutPlannerTool,
    LlmWorkoutPlannerTool
]

# Setup model
model = ChatOpenAI(model="gpt-4", temperature=0).bind_tools(tool_belt)

In [14]:
# Define agent state
class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

# Define nodes
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}

call_tool = ToolNode(tool_belt)

In [15]:
# Create workflow
workflow = StateGraph(AgentState)
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)
workflow.set_entry_point("agent")

In [16]:
# Define edge conditions
def should_continue(state):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "action"
    return END

workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("action", "agent")

In [17]:
# Compile the workflow
app = workflow.compile()

In [18]:
def pretty_print_result(result):
    if result and isinstance(result[0], AIMessage):
        content = result[0].content
        print("Workout Plan:")
        print("=" * 80)
        print(content)
        print("=" * 80)
    else:
        print("No valid result to display.")

In [19]:
# Function to run the workflow
async def run_workflow(input_message, view_process=False):
    inputs = {"messages": [HumanMessage(content=input_message)]}
    
    last_result = None
    
    async for chunk in app.astream(inputs, stream_mode="updates"):
        for node, values in chunk.items():
            if view_process:
                print(f"Receiving update from node: '{node}'")
                if node == "action":
                    print(f"Tool Used: {values['messages'][0].name}")
                print(values["messages"])
                print("\n\n")
            
            last_result = values["messages"]
    
    return last_result

# Function to execute the workflow
def execute_workflow(input_message):
    result = asyncio.run(run_workflow(input_message))
    pretty_print_result(result)

In [20]:
# Example usage
input_message = "I'm at the intermediate level, looking to increase muscle mass. Can you create a workout plan for me with only body weight? I can exercise 5 to 7 days a week at home for about 45 minutes each session."
execute_workflow(input_message)

Workout Plan:
Here's your personalized workout plan:

**Day 1 - Chest and Triceps**
- Push-ups: 4 sets of 15 reps (Rest: 1 min between sets)
- Dips: 3 sets of 10 reps (Rest: 1 min between sets)
- Diamond push-ups: 3 sets of 12 reps (Rest: 1 min between sets)

**Day 2 - Back and Biceps**
- Pull-ups: 4 sets of 10 reps (Rest: 2 min between sets)
- Chin-ups: 3 sets of 10 reps (Rest: 2 min between sets)

**Day 3 - Legs and Glutes**
- Squats: 4 sets of 20 reps (Rest: 1 min between sets)
- Lunges: 3 sets of 15 reps per leg (Rest: 1 min between sets)
- Glute bridges: 3 sets of 15 reps (Rest: 1 min between sets)

**Day 4 - Core**
- Planks: 3 sets of 1 min hold (Rest: 1 min between sets)
- Russian twists: 3 sets of 20 reps (Rest: 1 min between sets)
- Leg raises: 3 sets of 15 reps (Rest: 1 min between sets)

**Day 5 - Chest and Triceps** (Repeat Day 1)

**Day 6 - Back and Biceps** (Repeat Day 2)

**Day 7 - Legs and Glutes** (Repeat Day 3)

Remember to maintain proper form during all exercises. A